In [ ]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper, ArxivAPIWrapper

In [ ]:
api_wrapper_arxiv=ArxivAPIWrapper(
    top_k_results=2,
    doc_content_chars_max=250
)
arxiv = ArxivQueryRun(api_wrapper=api_wrapper_arxiv)
print(arxiv.name)

In [ ]:
arxiv.invoke("Attention is all you need!")

In [ ]:
api_wrapper_wiki=WikipediaAPIWrapper(
    top_k_results=1,
    doc_content_chars_max=500
)
wiki = ArxivQueryRun(api_wrapper=api_wrapper_wiki)
print(wiki.name)

In [ ]:
wiki.invoke("What is machine learning!")

In [ ]:
from dotenv import load_dotenv
load_dotenv()

import os

os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_v2"]="true"
os.environ["LANGCHAIN_PROJECT"]="ReAct-agent"

In [ ]:
def add(a: int, b: int) -> int:
    """Adds a and b.

    Args:
        a: first int
        b: second int
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiplies a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b


def divide(a: int, b: int) -> float:
    """Divide a and b.

    Args:
        a: first int
        b: second int
    """
    return a / b


In [ ]:
tools = [arxiv, wiki, tavily, add, divide, multiply]

In [ ]:
## initialize llm model
from langchain_groq import ChatGroq

llm=ChatGroq(model="qwen-2.5-32b")

llm_with_tools=llm.bind_tools(tools)

In [ ]:
from pprint import pprint
from langchain_core.messages import AIMessage, HumanMessage

llm_with_tools.invoke([HumanMessage(content="What is the recent AI news")])

In [ ]:
llm_with_tools.invoke([HumanMessage(content="What is the recent AI news")]).tool_calls

In [ ]:
### State Schema
from typing_extension import TypedDict
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

In [ ]:
### Entire chatbot with LangGraph
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

### Node definition
def tool_calling_llm(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


### Build graph
builder = StateGraph(State)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message (result) from assistant is a tool call -> tools_condition routes to tools
    # If the latest message (result) from assistant is not a tool call -> ools_condition routes to END
    tools_condition
)
builder.add_edge("tools", "tool_calling_llm") # <- This what ReAct agent architecture

graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
messages = graph.invoke(
    {"messages": HumanMessage(content="What is attention is all you need, Add 3 plus 3 and then mutiply 10")}
)

for m in messages['messages']:
    m.pretty_point()